In [4]:
!streamlit run /opt/streamlit/main.py

2022-12-03 09:58:16.748 INFO    matplotlib.font_manager: generated new fontManager



  You can now view your Streamlit app in your browser.

  Network URL: http://172.30.0.2:8501
  External URL: http://186.125.69.216:8501


  A new version of Streamlit is available.

  See what's new at https://discuss.streamlit.io/c/announcements

  Enter the following command to upgrade:
  $ pip install streamlit --upgrade

2022-12-03 09:58:30.399 Created a temporary directory at /tmp/tmp6zgprc5q
2022-12-03 09:58:30.399 Writing /tmp/tmp6zgprc5q/_remote_module_non_scriptable.py


In [4]:
!ls /opt/data/initial_load

ls: cannot access '/opt/data/initial_load': No such file or directory


In [65]:
spark.read.table("henry.yelp.testtab").show()

22/12/05 04:37:55 WARN V2ScanPartitioning: Spark ignores the partitioning ${p.getClass.getSimpleName}. Please use KeyGroupedPartitioning for better performance
+---+-----+
|key|value|
+---+-----+
+---+-----+



In [1]:
import os
os.environ["JAVA_HOME"] = "/opt/java"
os.environ["SPARK_HOME"] = "/opt/spark"
import findspark
findspark.init()
from pyspark.sql import SparkSession
from cassandra.cluster import Cluster
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
import pyspark.pandas as ps

/opt/spark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/airflow/.ivy2/cache
The jars for the packages stored in: /home/airflow/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-286ee7b5-b219-4ec4-8709-b7e9596b35f3;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.2.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.2.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central

22/12/05 08:47:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
from cassandra.cluster import Cluster
cluster = Cluster(contact_points=['cassandra'],port=9042)
session = cluster.connect()
session.execute("""
CREATE KEYSPACE IF NOT EXISTS yelp WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 3 };
""")

In [3]:
spark.sql("CREATE DATABASE IF NOT EXISTS henry.yelp WITH DBPROPERTIES (class='SimpleStrategy',replication_factor='1')")
spark.sql("CREATE TABLE henry.yelp.testtab (key Int, value STRING) USING cassandra PARTITIONED BY (key)")

""


In [6]:
df = ps.read_json(r'/opt/data/initial_load/review.json')
df = df.head(10000)

/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_json`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [7]:
df = df.head(10000)

In [24]:
more_reviews = df['business_id'].value_counts().iloc[:10]
more_reviews = more_reviews.index.to_list()

In [26]:
filtered_review = df[df['business_id'].isin(more_reviews)]

In [87]:
filtered_review.reset_index(drop=True).to_pandas().to_json('/opt/data/initial_load/filtered_review.json')

/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [31]:
checkin = ps.read_json(r'/opt/data/initial_load/checkin.json')

22/12/05 04:21:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [32]:
filtered_checkin = checkin[checkin['business_id'].isin(more_reviews)]

In [88]:
filtered_checkin.reset_index(drop=True).to_pandas().to_json('/opt/data/initial_load/filtered_checkin.json')

/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [38]:
business = ps.read_json(r'/opt/data/initial_load/business.json')

/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_json`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


22/12/05 04:22:55 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [39]:
filtered_business = business[business['business_id'].isin(more_reviews)]

In [40]:
filtered_business

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
141,230 Stearns Wharf,"(None, None, 'beer_and_wine', {'touristy': Fal...",SZU9c8V2GuREDN5KgyHFJw,"Live/Raw Food, Restaurants, Seafood, Beer Bar,...",Santa Barbara,"(11:0-21:0, 0:0-0:0, 11:0-21:0, 11:0-21:0, 11:...",1,34.408715,-119.685019,Santa Barbara Shellfish Company,93101,2404,4.0,CA
957,201 Royal St,"(None, None, 'full_bar', {'romantic': False, '...",W4ZEKkva9HpAdZG88juwyQ,"Bars, Breakfast & Brunch, Restaurants, Barbequ...",New Orleans,"(11:30-20:0, None, 11:30-20:0, 11:30-20:0, 11:...",1,29.954387,-90.068363,Mr. B's Bistro,70130,2064,4.0,LA
1971,2209 Magazine St,"(None, None, u'beer_and_wine', {'touristy': Fa...",8uF-bhJFgT4Tn6DTb27viA,"Food, Donuts, Burgers, American (Traditional),...",New Orleans,"(7:0-21:0, 7:0-21:0, 7:0-21:0, 7:0-21:0, 7:0-2...",1,29.929426,-90.077069,District Donuts Sliders Brew,70130,2062,4.5,LA
2380,2660 N Campbell Ave,"(None, None, u'full_bar', {'touristy': False, ...",UCMSWPqzXjd7QHq7v8PJjQ,"Restaurants, Cocktail Bars, Bars, Comfort Food...",Tucson,"(7:0-15:0, 0:0-0:0, 7:0-15:0, 7:0-15:0, 7:0-15...",1,32.255528,-110.943492,Prep & Pastry,85719,2126,4.5,AZ
4497,333 Saint Charles Ave,"(None, None, 'full_bar', {'touristy': False, '...",GBTPC53ZrG1ZBY3DT8Mbcw,"German, Restaurants, Seafood, Cocktail Bars, F...",New Orleans,"(7:0-22:0, 7:0-21:0, 7:0-22:0, 7:0-21:0, 7:0-2...",1,29.950742,-90.070416,Luke,70130,4554,4.0,LA
5851,1418 Magazine St,"(None, None, 'none', {'touristy': False, 'hips...",vN6v8m4DO45Z4pp8yxxF_w,"Vegetarian, Restaurants, Breakfast & Brunch, C...",New Orleans,"(8:0-15:0, 8:0-15:0, 8:0-15:0, 8:0-15:0, 8:0-1...",1,29.935799,-90.071906,Surrey's Café & Juice Bar,70130,2084,4.5,LA
8098,1796 21st Ave S,"(None, None, 'none', {'touristy': False, 'hips...",pSmOH4a3HNNpYM82J5ycLA,"Restaurants, Sandwiches, Food, Breakfast & Bru...",Nashville,"(6:0-15:0, 0:0-0:0, 6:0-15:0, 6:0-15:0, 6:0-15...",1,36.136985,-86.800538,The Pancake Pantry,37212,2063,4.0,TN
9506,2707 S Virginia St,"(None, None, None, None, None, None, None, Non...",PY9GRfzr4nTZeINf346QOw,"Event Planning & Services, Casinos, Beauty & S...",Reno,"(0:0-0:0, 0:0-0:0, 0:0-0:0, 0:0-0:0, 0:0-0:0, ...",1,39.497687,-119.801139,Peppermill Reno,89502,2486,4.0,NV
10607,118 S 20th St,"(None, None, u'full_bar', {'touristy': False, ...",EtKSTHV5Qx_Q7Aur9o4kQQ,"Bars, Nightlife, Whiskey Bars, Burgers, Restau...",Philadelphia,"(16:0-22:0, None, 12:0-22:0, 12:0-21:0, 16:0-2...",1,39.951417,-75.173768,Village Whiskey,19103,1553,4.0,PA
11426,920 Olive St,"(None, None, u'full_bar', {'touristy': False, ...",M0r9lUn2gLFYgIwIfG8-bQ,"Ice Cream & Frozen Yogurt, Burgers, Food, Nigh...",Saint Louis,"(11:0-21:0, 0:0-0:0, 11:0-21:0, 11:0-21:0, 11:...",1,38.628864,-90.194542,Baileys' Range,63101,1567,3.5,MO


In [89]:
filtered_business.reset_index(drop=True).to_pandas().to_json('/opt/data/initial_load/filtered_business.json')

/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [45]:
tip = ps.read_json(r'/opt/data/initial_load/tip.json')

/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_json`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [46]:
filtered_tip = tip[tip['business_id'].isin(more_reviews)]

In [48]:
filtered_tip.head(15)

,business_id,compliment_count,date,text,user_id
315,vN6v8m4DO45Z4pp8yxxF_w,0,2017-06-12 00:46:19,I love to get carrot juice with ginger added- ...,prai-3PY4t4DX7l9R9Ek3Q
464,PY9GRfzr4nTZeINf346QOw,0,2013-06-02 17:25:59,The spa is outstanding! Very beautiful and imm...,JUR94o7gvS382zEzVizkHQ
947,vN6v8m4DO45Z4pp8yxxF_w,0,2010-11-24 19:11:18,Go during the weekday to avoid long lines!,mr7XKwztNhQ7e1T3zbcRGw
1028,GBTPC53ZrG1ZBY3DT8Mbcw,0,2011-04-29 17:33:43,Bad start. Rude staff. Hope it gets better.,Ye-9S8f2dt41UBfy4lXffA
1077,PY9GRfzr4nTZeINf346QOw,0,2015-12-31 20:03:25,Here for spa dinner & New Year's Party.,vTprcHGBn9asRfI349M1Fg
1505,GBTPC53ZrG1ZBY3DT8Mbcw,0,2010-06-16 16:54:18,Burgers are good but too expensive and the bac...,MUzkXfPS9JaMgJ907orz0g
1939,SZU9c8V2GuREDN5KgyHFJw,0,2015-01-11 21:54:09,"End of pier. Free 90min parking on pier, $2.50...",w5bAI7USuCvyl8kZzj6sQw
1992,PY9GRfzr4nTZeINf346QOw,0,2013-08-18 18:17:38,That's a wrap. See ya in two weeks for the bes...,BBdRlIK_eeFhdl4EL1pLAw
2524,GBTPC53ZrG1ZBY3DT8Mbcw,0,2015-06-20 20:07:58,good food but bit pricy,N_ppDrIIvtFk7_86pu42Dg
2613,GBTPC53ZrG1ZBY3DT8Mbcw,0,2012-10-06 23:38:39,Wow! P&J oysters plus happy hour! I'm a huge fan!,IwknZ7P0VWHSxOGsf4uMxw


In [90]:
filtered_tip.reset_index(drop=True).to_pandas().to_json('/opt/data/initial_load/filtered_tip.json')

/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [56]:
more_reviews_users = filtered_review['user_id'].to_list()

/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_list` loads all data into the driver's memory. It should only be used if the resulting list is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_json`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [2]:


user = ps.read_json(r'/opt/data/initial_load/user.json')
more_reviews_users = ['qEEk0PuoH1dVa619t8fgpw','EBa-0-6AKoy6jziNexDJtg','ZGjgfSvjQK886kiTzLwfLQ','JYYYKt6TdVA4ng9lLcXt_g','pitYOVSsF8R1gWG1G0qxsA','8fPlzYWo0j_nQrJMeyF0Fw','TGgfqWnUaCf6DM7TLuNhDQ','ZaMB7VbOwaARjxdhXjODxA','QJ-ikvhuRcigSCAWJTrnqQ','1xS8Jj23zHx8axIVopG3wA','Zn8uhC3DjoKjjwBiuM8oQg','ftRgzVFzv6-TOCBXEOdWeQ','VMtyZjaEJB9nfmjr4xdVlw','cARxOd_5yKCgsCbUZ5ED4Q','BBTexIhkFIYnS0rd56vsKQ','bme7nh1NwRfF4U34TXFK2Q','HVl561l9Y3jwMKNKnoxOyg','LcqNuhqaYt5ekKzaRirmIg','_l0csyXqNIcb3vG-1qR8DQ','E1olskL76b9TrcClgD9oBw','xjQSpme1Z7Xw8XehRLpYuA','86Fgk1s0tAVioJtVnJeBHg','XeS-0ONS5uoR_OfgZQebrA','Q_ixtOGUKUbCQ_sXThQAwA','cWAKzsMt0iMjBNPuNJVgDQ','n2pX5Ae8xCUi2_WlwcTkXQ','RHpCcdTbnBkgpdTYvbbTcw','0WqEkKMu03irkMiEtsFxZg','0MeivhX0kZCfV3zMtHtk9Q','afX9bbJ01xYU4L3qKbACUA','v8aeoMqdrClqfZxNB4lWCA','A9Q-_QpJy1mHjlwwP2RwJg','KLS_AWthM9n6KLcBTCF_RQ','GU_4tHnCYE6aKGU2XH-zkA','INUDMj7EmrLlTh6qDNprAg','7ktyPHE-NGnWxarOqjIQiQ','1Y38tVMSPH8jwIGw_y_E3g','I-wqWdMhZ-cgT4YPxY20gA','tt2qIFKZumubxk_UwXhijA','uJM_dR32MnrPztZZaC1rSQ','5BL_mcIfYvU9L08syGCfEg','c8qFkI_VusWo0xZvkjfBWQ','XuadFePn8P6l5epbEFnfKw','CJ1qOThxVs_V6VTYRL4Eaw','v62bgesQ-CSGDS1rNqXZsw','wKmJrv-p7gI6kBvlXQbcFw','Ebp-eYVK8Jtq-toYAHdDHA','UM1ApB4_5GJcxfaYS8jD4Q','8RcEwGrFIgkt9WQ35E6SnQ','gaqPcK4kIN9_N7htRQhTtg','A3gMFX-PQNR0aomxaDGJkw','yYTykS0TpVvhAtm0tda1sg','ij0Mu58AKlflicmBhI-xMA','mp3GeUkEr1gmMV5ZThtBpA','kcnmjaBiItk7wqk9kLiupA','X8tAoBJPDqEhwwNuCQVa0w','dQeH3N509e8AzzoC9Xdy5w','wulZapNwEvd8JA63oxTVKw','vIr2vYWApBXiQv5qHFn6gQ','bdIueXy-Sc0DfqzXbm9AEg','nc0pxjUX3IUM2diqr5ME4g','SQtIhJLfyPCSQvoA-qy4uA','Z_9WJOW0l_4jROfkkQWT4g','3fsfZqiRJKjEAiLIBbxsWA','sZuL9iA687GMN6AY3XBCIQ','d82e3N5c24CDA9Svi_t0fg','cHVfkm_h_vKt-ZIWMYQcrg','kFQ0u8c6x76jTwmEPmKoWw','6FVPYJedLt3FTCQlamuscw','1NOyCzEPeO6D7_2zHztOUQ','qVEOVvc9gB29v7IAWsnA_g','mkEqJ61lS7gJUR6nqlbRkw','1o4iynPusBLozi0CpLGH5Q','mgkPEU9hRPb9WnDwx3Z5iQ','8Rb0XNjTenrhJgLR2ydStg','lbj1atSiHxGFrzN9WAeynw','FV3oC7l8P_JWKULr2tnLFg','z3nv7VqIQEm8I7MTDwM-Ww','1WE0E1ewe2ZZU0l3V5vNWQ','mzdYdUsMk1uzl7kYyF7HaQ','94ICh-yDZXOOGu-e33C6RA','mqCL4lkyQLiNXj_74-tMvQ','nFiYurEbPZBAOCURMrBgag','jYknm57qpeZzTueiVIYTOA','K4myo2_Z_2KKdORpfdKoTQ','a6jqAsvBRT2mCkLhNjEY2g','vKGevla8lDHp8yxq4Poyfg','Hs4HViBxhjOkUJlthNKNWQ','uinzvCuEMwTRPcmgDtlAsg','KGIBBWFmyxnsc_-Ni0knmA','yrAX4ikJePO6eSN6_g6buw','PCISmI0atmUGCauThxGkPA','HYlb7vXGuiTvR5eiwSLBoA','leCO_9EFERggbuUoLYZ4gg','EmPcdONs6821YhkRQ8s87A','3zeNqvWimKiA9gOSbx2e6Q','hmVE7u0Nt45j5sZCH9j_YQ','uPdFJ22SxG74yhsatFVvrw','gcFwdhZfdhIacWQYbaLcBg','VwGzHDCtP9aZgXrUmgAajw','H88uWBzHemj6ldYJKU9-PQ','JclBkdNeLD0hpv9AR_HaeA','IwknZ7P0VWHSxOGsf4uMxw','JQe6xrPnYWZ3rWDbgwWZsA','DWASYCVz1yN6ytAFzfgEMQ','VZWt5nqlsjMntB2Ga97H5A','D-hz14YhfL_5d-BhcD02kw','HTFyT2aSIUUNYhw6fR99og','--4AjktZiHowEIBCMd4CZA','M9L3vxwilf57pxIk7xlQkA','BlU_lOa_o4FOG5wlvxRWpA','nXF3s0JyuSkxc01GaQLvOA','yU5TuZiKiG7VTgNGlffsYg','kaa6i7IHMyYIom7DIul9EQ','3A_Al7CYUQoLxK4CtnyabQ','5u5tto7PC3a7FQhIWEXflw','zq2nd0H1dVzi3bta4Zepuw','gbt3Ft7YmAmaxc8b27QM6Q','kQIp9HBSCjVfoGGV2wPLQg','MpBDUJXrry00iPnX5hbQGA','k6GurG0wmHVyiZGzmxs8eQ','KfPVXIzWw34sp9l5pfBE5Q','DQx7XI6ZxubfMndzFY22zQ','Vcbxj8JlIXkWgdvroc1qAg','6-DBwoNPw5zenYYSg6RrhA','WKJoektrmcMssSoQSI59ow','V0Kg9fkgfoCNvYpNJcv0Ng','hPuQIudnLHBJaJL7swpB8A','VZSLr8PCzqeh31HU8D9hXA','xKfuwsPWEaJwQiGaTYHelQ','Fq5CvUC1AzK8GSmpyn0UeQ','Khecid8LZRq9n6xe4qjgLA','h6uPago5wJSV4nPBO71iRg','IDKqeErl2l-huYugUzea6A','RGOJ68eoobRsYW4LgQtBcA','DGJRD862ewpN3Ux2pkZkHg','3PunMPL-ne29d2Oeq2PpXw','y5wLD2bvZ91fZnNid8Yh5g','ApOMEpkxD9yJTaaTxMfttA','v5PBVjPxj_W4-x89Lbc-TA','GQr0P3dgOZGSM3PYPoEtOg','HPlUnYMqvb5sGmX_paxCNQ','HXKZYi1d24WdafgmKhVUow','698q2zdsA2i9fYN1YhX1Xw','2anfbsmeoqeMYCiNwUow9Q','U8WHL_eN_XYntCINBrjThg','QsByY2gW90viW7wiqWeEcg','gyAfijhxmJzvwxPD9y6RHg','oh1jk_g-LqpKybskPYoP0Q','DtoCCg1vpf_Mj5Qk63zk3w','_w3Zqx8DiAEw6hT980Upvw','vxw6ilzQ9gKIfxLP3GvVrQ','eefUAeYwjVLTDu1iRxn7Iw','4gBvBdTybB8yydxh1nhOeQ','dhQOA8-FdpA6D0hNhXEYIw','Ytxn9vgPaA6X_1dKO_WNSw','SUV5QErmlMLhwsIkdUzK4w','TS_3tnBpHnlBYAoe978Xww','RbT0tdUrDiazlTGaORWGhQ','RsI5Sjjv1-EYue9yVN6WOw','hO0vrPqf4Ma2foebnR44Iw','bSRQp1PBDzCKyvepEQSYxQ','GrCKTYu9qPiUlowByh3VAA','2_1YJigBXrIll1fjVDS30w','eILPNE5BR9yaS1R3jlks9w','8uX0U8FyL9-HwNa64m1FMA','IjaFggQWriqk2rWv6RpPvg','IzkWFIoq-8k4wX_hV71Qbw','3iRkz8HR7qorxpx9bifywQ','QIogFooFd7UTIEx5iIU5kw','b7Ab6SBEvffhOX836lxATA','AjdePuDfMJTHphuJCQBdSQ','XBNtqhc72X57VK8GLNbsGg','_rWCX5E8R_3Ufv9JetOtHw','H6sRbrU6hePN89irFWBKew','tekHDsd0fskYG3tqu4sHQw','AxLCLK3Hxv-E7EntE3Y5gw','MxlkrZRxrSdJsPRdg1521A','ua3zAwmTMGvWdBR_rrjuzA','7BtVy5wVDth3-lwFbUgDXA','1GnIzms3F3MJJK2kyIwuHQ','y9dE4Q3D2Pu7dvVGdd-9Yw','UrKgCQvSTSUay1GNCkjnhA','HSQWc-Rgc_OpUTdKmSLjkQ','5UJ1ToXJ1Q4hpINbHcOJSQ','TA-wRnDB3Xv7_pSajJIUBg','jDWCDWUujfFpChAX8rgWYg','mPE3PKQ947F92s-4XvhQag','6UEJvMhHYIDzXI51lHPiIw','Usrg_bh_e7vJVwd0EMvkOw','a4WIm73rjfz1Z9_QGCGmaw','h4B9ZDALqcGev5XPyZ-Pow','iBQKwkuDvAdTM5gLWHgZwg','W1b5tQZzFN5KinOwypQDig','aDWWdGPZTyFkMGNyEwaVZg','bdfx_mqNeiYm-UKAkgRnRg','TBRQIFGGWjR7Z5Ki5_PfxQ','1-SMe3meQDCUyOypIS28PQ','dod4blQ8cwm3fdE00YFC0A','QBUFj_B6BH3QKzZpYHJz5w','zqCiUyWGnh633-2vLmuemw','88Ij88GMC7tcIegOicRdYA','4_IJKLhoyof73pvRal4Exw','vupwwaLgP0sVb6Qrc3Arvg','2kH_cKV0s_uZEIlCoHnzvw','W5fchLNpH6GmiVxx24zXgg','Sa9Jbde7d2kuzhVEAMRWEg','CfhitdMbQ88r9nlalyEb4w','Mv779di54QwcI43AO2mPLQ','xLcwDhFKONBP2F16lsaB_w','29UB_wrnUIdsxV2ZmrlZSg','zF_unBFpdWMY6el-B7wO8A','K1Cfhs2mESLD1nUWWfkFwQ','0qaUsn4TGgJYfbSxg_Fp2A','aAvl82YfsPmnNOMSMD8jBQ','Xjfc1owyGoFcM94kRxOCJg','fvgyz-tffn87IqaVkkN_DQ','NYxNe3Q5ROXBe7Fatst52g','GdcVn9hdpRa_rlYiEFaN3w','8i0Do65MQwRG4HAzO_detw','5i-nwtUBzji5An5fr4cpgA','m1q4gB6wdawoQXjPbs29mg','TDCb1p7KCiOd71tkY2rJ5g','4HzAp_Ck-hNIK7f4hEyKXA','BeS9uhpmc6Icap_Eso9Lqw','G3UtIVC0Sl8cbiA0_cPyMg','MvjdQ7T9YiBC9bL9fMaN3Q','ywH87wgOyI6ttRh0EgEeHg','CNmCBh1WQs-5a8RwIq0sMg','91z9RokpfMdwbxkdAA3-nw','aVwveJmgYeeN-ZnDX_shBw','IAReqg4gfz2OHz84p-qQ2w','ey88ywOdRVGEMmd6YyJNIA','tCti1lZLnkt5TXdOEdXqoA','7QTh-fkw9Nr2lO10-PV8yw','GQgH--hxKJisGmfsKj0xDA','zfvjfJ3RJR80A5D81emi4Q','NVxCOYSTlcRpnxaGop9N_g','Wjlo5YcDe2Lc7I4WNqooDw','7SgM5DrVSfkaqCpyeKjT0g','GlprrSVlHYCjhQtK1_LB6Q','337n8uBnr1pEP69JzVWDCQ','ft-zQNrmSXpG2heoTorC2w','UgYUeboBoETAgxom9iRdJQ','E_SoAPv9OT-eEzDZPoz2Tg','fQ2H73hk2r8vTeh-z7EhbQ','q8lvniKIxLt-d9zQ4yeEVg','HHkcBGeuKN55Ka5aULarmw','YqZniLami3yOXF3CSVxaFQ','7ziWZULyiZv2TesYNMFf4g','zDj-OYDs0dHjg2aXcVxD9A','IE4xKfypWuM3eaaq5q9IxA','xKsTgilmON3UrhEpM5Zl2Q','5Q5aeMq_dgomcVaSuvN5Lg','3R43ydfitOG0B4e_vuQyfA','6wRHe58b9X1mP7C86agykw','h59iX3YOfP1laPF000BV0Q','UnfeDunSlKcxvy2LPlzYBQ','AVNQyhynAB5z6iuN7P3Jfg','4LbxeQIyej3mr6HYYTsd7Q','WQ-_hGAsrAeR9QrZ4HH2lw','mDFeDwPLtG2TERrsR9kudw','uR4wrcLxQ7VUGBjM00iHNQ','5idrssvX83bcWhz11o4e1A','xW2A0MciHB0pLB4RHTi0nw','Q6vpjOTgGziXBkyURnUOqQ','wiIfZ8T2JdsRb8OKmdQj0Q','S7bjj-L07JuRr-tpX1UZLw','Nx-pMoHLNgB9ZVE3vBP05w','4reeeVHHWyZOp_Ejq5n40g','mSyINFt6TtXX9OHbwJfN5Q','yBkMMjXlQCEl7hiIGhQYgw','8Egic8Gg5qH0zC0mzB8BQA','rPzhe3wPt_68L3wrEhFg7g','d7qDp1KKg7GOLUMJLrlUsw','SjiTI7mX1UKdJbPLYeGI_g','3ywIG2FLBr9JNp6dnc8u3g','dEfQnSNy5US5FT1o5CMRbg','DaqsvhFFWRPMdLkriySHbg','acgJkn8BuKbBiKK956mWZw','LqC5hNXcFVY-jcbyxi-tuQ','GltpBXG2dCiMvi-bxvJPHA','1PhM0nRN6WN6bZadHN5cWQ','srou4DkAZLbVsgAPGqzN-g','ppYEK9zY7Vlhdf855ehlHA','C7hvShEj1G--Rc0exx6seQ','PGkMTm3hiermfR6TE498bw','LQ2CDCzIkXrUMXnh-W9QSg','RUQkywD4HQv2hvfv69Qq5g','bdalWswHTzYC1RDBYLehgw','TMMXLS-gEUQYDirHhL-9Og','yf7Wj4KMHh4Jv8Jwir8Kgg','QQwcp_Fmv7I3PLDAgUP1Mg','Oo_lq_YEKYfTSIO-1fnQvg','M6oZKl57_5bz0CeCCADdBA','-LpL5lWFDRytw5PADY1p5Q','P26U3x0gM5cG1s4gvWM82w','1Twew_ar_EPpb-NHSfcrog','azJSQWKdhUCsFELul6sVnQ','d5n7YWztGkm36c2LBQ8Y_w','oZ7y2hZHfnaoucaZC9Hp4g','NfjVkSRNironQBK0kHNoJw','DUNV27mVp5ATW6-J9LOJeA','8NjgVWDhLFNOTy19wJBoig','nfMgsrqADr_94KitluyCCQ','Du6bF8BUoJsIAuC0J3KLIA','xRJyMR1_QPuZuUhUuVaooQ','sMcCc4hcmnr_yolsz80gjA','mkQz8aihI1V19sYH8JQy-A','xSlBsTTPtPvwlclBIvAjYw','AO7thy9_PAXAT3rze_iaXw','c1XHqYtBl5qyeAspzPiCFg','sxGxgtlI6ipSimdUlSC0Fw','HmmBrh2_jdngGxjwzzzmdw','NiJmVx6JPcAUoE3d4j9aVg','ljqXU2O8o1Ofuu_CNRnXxA','RuYYtPAdhVjQ6u6GqmgZHw','6eErC2_WqGtCMO1tgqT1CA','CaSCl3j_tXFPhF6qzQciBQ','npaqVmBLp_bRk4DcraZYww','b9Q74D4XvFjJHKvLBLSaEA','1VJHfBEGw5xaEKcVSr38kQ','G35tigeQvOPqRVMzJVY9IQ','d-4Diz5jY05LPUE0mCBXlA','KJIzmEQ6peS6yc4RjlKS4Q','MVfbSEGqvhDWdmAfUeG38g','tNq35k92WAkSLSqeof03hg','aYQjCpAWtZMFS_Ce2ogvzA','CcXSCGzlhN8Of6xwafPkIQ','i2pV48tnyrf779rPkjnilA','H32QTRV87Q6CGTVUm1Z4QA','M7C_aSxzZyk93KNUmPUpYQ','k24QIVcmhhOnv9MCY3dELA','zxeXnjqmlrAspfk17LSZCg','mpnA1B30uMYGeuvKbJJgEw','6hNft_Qgu7T3b1X9B1-K5A','n1dDZfXcqw2A3iNwU5xlzg','WAzX3AaM6PQfzFJHaUF6xQ','2gWG2KZEhyIXXy5SOiUOZg','5BpTDdcXblUeG2dV-xciWQ','JRsGihBLWYVeeWRLP3in9A','qde7cw7YIJutTmRiFTEmfg','gqDbe1YwS4EhuCCNSNhpxg','mprkVxwLfSj6gW70USN7xA','41TYTuV87wmkLe-nrbwS3w','_1pVMv3nA38y5V9Cz8Zu_Q','QxVm4V55Dtdfv9Zfwm4daw','3dWsbFVG1c2iPzBM-WOQNA','39eSgXJRZwdWHs0IDAtzhQ','ovybrUwHdWRbFlKSM5H2Yw','kE8t6sP0yc4d47caMOyifQ','9IW9u_N2FfV-y5sUmLpYKQ','9ZiofeADDh_whZD8I9KDFw','g794-dhjC61kWSn1l3PoQA','-F6PdlUlQMGrsX-qug_f4g','KkmADj2xPhJL6DbZRVQcLQ','7xvUccpmp1fXWLAkZaXkDQ','G6ZnatT96yzdcX81PZyT3g','VEWzJnleYqTOVglmDALEyA','zFVBIOECiSpDb-e1WZOFVw','vzNYQaFyBnIU_bBkJsUJQA','Ng7RCiL-EjwAZo2jZoCwkA','5dXjCqtB1P4U3u6y8eEaZw','__1iEm-E3tvKuToIOCb76Q','IhM58htabC6OfgOdDyEW-Q','aD-xli-bVq6ABWuC_OqWIQ','riWDgbauId8TK7YFVgMNJw','uh3QEknKPHRG1wVABRYqWQ','3dcFnCZzBZrcKDeIYgiScw','J0Q7h3aSp4y1ElwA_JRAJQ','jlG7lVsF5LY2CxQUCGeQKg','0lSEYfAUQCqyBIaEQQUbbA','1OY-70r-JQaLbRbRBm04lA','9OJdZbS-8DiEtzqicUYQ8g','5dWx3ZbicVHtW7yGndsRoQ','4a7y9WOKYJmx5Hk_aUhmgg','i0FuzL8Aa5wV2O-n59_pEA','9woNV9FkJMX4MHygee84vA','BacuN4z2jKKPyBYV_wnAcA','xZ5TBpdWkHH15WTDgAqYFw','jheopFKRPEDAR_ur5HGTug','s16-BUo-orUsELvMu5ocKg','Wrdbwj4kmvlpptA0-ErOdA','yvQty5DgFfXlzZjnXteVbA','sruMjGMA3KbtDlMNvhwRLA','-G-s0z4MVaIjHIRAWp1myw','h8o3NmCcg3BAIHsBsm32Sw','xU5bD7aUKb9-LulOeBxCmQ','LrTOhEVai01haBAIMUY7mw','LimLRJP6n8pDPAT8knRhyw','0RJcckForPWsnVNMn_orYA','L8nFSWzvcsvLKKEekT5GuA','f2MiAt7rHnBq7LPL_9iqvA','zn_biH8_d_2j9wVgXj5Fsg','00hnvyIyIheNoyzhcOLZUA','A03MjVYocHO-6xOuk59ssw','WmfhKlnkeDYXwsY55xSO0g','9jV3AfGkcwPZNB1zR3jqbQ','Jd-ccxgw6bPWZMpiNNGijQ','XPzlQJDXbIPn71Hsgykugw','_l01jFciQKtGR1BuvrMG-g','ZnzpvxOa7eiLESrebeufJQ','q1MxLnDtvth41eaEogCeHw','Q7uJYSicQdI_RZbG8kjnSw','UYU71DRx-UDAee-eFOFfWw','R0w9KhptxXufhSt8dMNqnA','c1U0sTwSFFxShqB3leLoWA','0TL2UvlfvIe6g53JhjpiEQ','RNI2SJkPGW9L4Wm2YSxcMw','_XrNNcEbNnut2Q4Py-LfKg','60w9x_k88BtoVjwGLFn6nw','Lk6yr0Qff2ND7gzU075PRQ','ziIKYrsEnWOIogJhYJnVng','b9HV74ZB2neuy51hWstUGQ','dbELeRFW0v_bXybQNfb-_g','z3pethjEj75TLrX66FhUFQ','PCDorKmP5__w3BkyzjA2Xg','9UWf_0ohtl3_yqHdUjUe5A','wRfQaWiGKiR_htiSEkauSw','8z3dCuY4wurac7jhj7Oxew','ZOsDHReu1-GZuGgm9CeJQA','Rj88JTogoHMKg0wy9t3vhQ','YfvysNidwMsQtw56XAnGdQ','h6a1CAlAN-vm-JgwUoWeSg','adh20lqqWiv_cNhDfkPkNw','W9UO0ghofZM4WrFws44nGQ','v0JgXAAo2a1v5X2rSIpIPA','fkE8bjLL-fxYj-1_rlq28w','k6zYbMle3LI9jhMuY4Tc4w','q34MA5yPVVEkIV2mldl0jw','EEJZi7OjIY21ec-KXDa15Q','ySQP6km_KmfSQ48h1FxqLw','26F2byB18-dq7319tJHXrA','MksXytmVtf3oELjYMBlJ0A']
filtered_user = user[user['user_id'].isin(more_reviews_users)]
user = 0 

/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_json`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [3]:
filtered_user.head(15)

22/12/05 08:48:49 WARN MemoryStore: Not enough space to cache rdd_6_5 in memory! (computed 41.0 MiB so far)
22/12/05 08:48:49 WARN BlockManager: Persisting block rdd_6_5 to disk instead.
22/12/05 08:48:49 WARN MemoryStore: Not enough space to cache rdd_6_1 in memory! (computed 47.5 MiB so far)
22/12/05 08:48:49 WARN BlockManager: Persisting block rdd_6_1 to disk instead.
22/12/05 08:48:49 WARN MemoryStore: Not enough space to cache rdd_6_7 in memory! (computed 54.6 MiB so far)
22/12/05 08:48:49 WARN BlockManager: Persisting block rdd_6_7 to disk instead.
22/12/05 08:48:57 WARN MemoryStore: Not enough space to cache rdd_6_3 in memory! (computed 42.7 MiB so far)
22/12/05 08:48:57 WARN BlockManager: Persisting block rdd_6_3 to disk instead.
22/12/05 08:48:57 WARN MemoryStore: Not enough space to cache rdd_6_0 in memory! (computed 99.8 MiB so far)
22/12/05 08:48:57 WARN BlockManager: Persisting block rdd_6_0 to disk instead.
22/12/05 08:48:57 WARN MemoryStore: Not enough space to cache rdd

22/12/05 08:49:10 WARN MemoryStore: Not enough space to cache rdd_6_6 in memory! (computed 48.4 MiB so far)
22/12/05 08:49:10 WARN MemoryStore: Not enough space to cache rdd_6_0 in memory! (computed 40.4 MiB so far)


22/12/05 08:49:11 WARN MemoryStore: Not enough space to cache rdd_6_9 in memory! (computed 3.7 MiB so far)
22/12/05 08:49:11 WARN BlockManager: Persisting block rdd_6_9 to disk instead.
22/12/05 08:49:11 WARN MemoryStore: Not enough space to cache rdd_6_3 in memory! (computed 3.4 MiB so far)
22/12/05 08:49:12 WARN MemoryStore: Not enough space to cache rdd_6_5 in memory! (computed 18.2 MiB so far)
22/12/05 08:49:12 WARN MemoryStore: Not enough space to cache rdd_6_1 in memory! (computed 18.0 MiB so far)
22/12/05 08:49:12 WARN MemoryStore: Not enough space to cache rdd_6_7 in memory! (computed 82.0 MiB so far)


22/12/05 08:49:16 WARN MemoryStore: Not enough space to cache rdd_6_8 in memory! (computed 88.3 MiB so far)
22/12/05 08:49:16 WARN BlockManager: Persisting block rdd_6_8 to disk instead.
22/12/05 08:49:16 WARN MemoryStore: Not enough space to cache rdd_6_13 in memory! (computed 9.0 MiB so far)
22/12/05 08:49:16 WARN BlockManager: Persisting block rdd_6_13 to disk instead.
22/12/05 08:49:16 WARN MemoryStore: Not enough space to cache rdd_6_14 in memory! (computed 9.3 MiB so far)
22/12/05 08:49:16 WARN BlockManager: Persisting block rdd_6_14 to disk instead.
22/12/05 08:49:16 WARN MemoryStore: Not enough space to cache rdd_6_12 in memory! (computed 20.2 MiB so far)
22/12/05 08:49:16 WARN BlockManager: Persisting block rdd_6_12 to disk instead.
22/12/05 08:49:16 WARN MemoryStore: Not enough space to cache rdd_6_15 in memory! (computed 12.1 MiB so far)
22/12/05 08:49:16 WARN BlockManager: Persisting block rdd_6_15 to disk instead.
22/12/05 08:49:16 WARN MemoryStore: Not enough space to cac

22/12/05 08:49:23 WARN MemoryStore: Not enough space to cache rdd_6_9 in memory! (computed 19.4 MiB so far)


...

22/12/05 08:49:49 WARN MemoryStore: Not enough space to cache rdd_6_19 in memory! (computed 49.6 MiB so far)
22/12/05 08:49:49 WARN MemoryStore: Not enough space to cache rdd_6_20 in memory! (computed 59.8 MiB so far)


22/12/05 08:49:52 WARN MemoryStore: Not enough space to cache rdd_6_21 in memory! (computed 52.9 MiB so far)
22/12/05 08:49:53 WARN MemoryStore: Not enough space to cache rdd_6_23 in memory! (computed 27.7 MiB so far)
22/12/05 08:49:53 WARN MemoryStore: Not enough space to cache rdd_6_22 in memory! (computed 54.0 MiB so far)


22/12/05 08:49:53 WARN MemoryStore: Not enough space to cache rdd_6_24 in memory! (computed 47.9 MiB so far)


22/12/05 08:49:56 WARN MemoryStore: Not enough space to cache rdd_6_0 in memory! (computed 40.4 MiB so far)


,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,compliment_profile,compliment_writer,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
276,3.65,80,14,80,38,4,11,98,12,137,6,32,610,"2010,2011,2012",18,"nk_8NpN5KxSsrXxfFiKEQQ, IVpM9QGea_sSDm3uortosA...",609,kathy,443,1156,_XrNNcEbNnut2Q4Py-LfKg,2010-01-03 00:32:29
547,3.71,152,2,152,40,8,19,86,3,330,13,103,1891,"2010,2011,2012,2013,2014,2015,2016,2017,2018,2...",96,"5INJa2npTkjUkX75ptn6WQ, mJCK1qpsCkKePozYIrIT-g...",1491,Charles,897,3214,0MeivhX0kZCfV3zMtHtk9Q,2009-01-11 04:34:11
1079,3.69,54,1,54,13,4,5,48,10,44,1,54,1507,"2015,2016,2017,2018,2019,20,20,2021",103,"biZ0fIcZp7B-KBJ3EGcX7Q, RTW5MOYCTpeyEHA6TkOSKg...",959,Shannon,1853,3694,H32QTRV87Q6CGTVUm1Z4QA,2008-03-09 02:12:31
1220,3.85,25,2,25,22,1,9,19,11,33,7,23,515,"2012,2013,2014,2015,2016,2017",62,"QMo7BDncb54YJVKeRHx6zA, eHAqpUdvRwfj-J02YmgJ3g...",342,Vicki,406,1106,3dcFnCZzBZrcKDeIYgiScw,2010-06-03 03:11:57
1395,3.99,24,0,24,7,0,2,18,6,15,1,3,154,"2010,2011,2012,2013,2014,2015,2016",13,"34ZVYUrYaNFBKGTtaYNUNQ, fn_cob62tSLGJh7lJImH_Q...",73,Angel,328,425,k6GurG0wmHVyiZGzmxs8eQ,2008-09-10 20:40:25
1462,3.61,3,0,3,2,0,0,0,0,0,0,3,23,,4,"tduMSflg1bhAFDgWOr4MPg, q2_A-WEmw5lfBQ-RhPQp8Q...",35,Shela,31,38,h6a1CAlAN-vm-JgwUoWeSg,2012-04-09 03:15:41
1556,3.93,0,0,0,3,0,0,5,0,5,0,2,18,,3,"XmFS3qFG8MxiFEQJeK9bIQ, KDewJBJknfWvQyHHFNbfdg...",15,J,57,77,WQ-_hGAsrAeR9QrZ4HH2lw,2006-09-05 02:33:16
1745,3.96,6,0,6,0,0,0,2,0,1,1,5,32,2012,3,"P4WR5ft0l30RJssFvSxKyg, fNB-BGLbY2XzASvHHufKJg...",20,Cay,95,147,bSRQp1PBDzCKyvepEQSYxQ,2010-02-23 23:53:26
2098,3.82,112,10,112,91,19,28,52,5,88,13,136,2391,"2010,2011,2012,2013,2014,2015,2016,2017,2018,2...",255,"52oH4DrRvzzl8wh5UXyU0A, E9Bmw3ZjTwiNukC5SOY_Dg...",2267,Josephine,1881,4139,s16-BUo-orUsELvMu5ocKg,2009-10-20 16:22:12
2574,3.88,4,0,4,2,0,1,3,0,0,0,0,46,,1,"RjZnvBLw9_Q_QuPUIVzVNw, iEQHAu1sJIkjAgaB0eh1JQ...",17,Larissa,24,77,aVwveJmgYeeN-ZnDX_shBw,2008-11-08 13:03:46


In [91]:
filtered_user.reset_index(drop=True).to_pandas().to_json('/opt/data/initial_load/filtered_user.json')

/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [68]:
import pandas as pd

In [75]:
filtered_checkin

,business_id,date
20627,8uF-bhJFgT4Tn6DTb27viA,"2013-10-15 13:28:34, 2013-10-15 15:13:25, 2013..."
32983,EtKSTHV5Qx_Q7Aur9o4kQQ,"2010-01-16 14:39:27, 2010-01-24 02:04:33, 2010..."
35556,GBTPC53ZrG1ZBY3DT8Mbcw,"2010-01-30 00:54:16, 2010-02-17 23:48:12, 2010..."
47397,M0r9lUn2gLFYgIwIfG8-bQ,"2011-10-16 00:14:15, 2011-10-16 05:09:48, 2011..."
54559,PY9GRfzr4nTZeINf346QOw,"2010-01-16 05:02:18, 2010-01-17 02:48:34, 2010..."
60796,SZU9c8V2GuREDN5KgyHFJw,"2010-03-07 05:23:49, 2010-03-21 05:07:50, 2010..."
64219,UCMSWPqzXjd7QHq7v8PJjQ,"2014-01-03 18:29:25, 2014-01-03 20:36:14, 2014..."
68044,W4ZEKkva9HpAdZG88juwyQ,"2010-02-28 17:04:54, 2010-04-16 23:26:32, 2010..."
109940,pSmOH4a3HNNpYM82J5ycLA,"2010-04-06 16:10:12, 2010-04-11 14:13:36, 2010..."
122182,vN6v8m4DO45Z4pp8yxxF_w,"2010-01-23 17:17:10, 2010-01-26 19:14:37, 2010..."


In [79]:
a = filtered_checkin.reset_index()

In [86]:
filtered_checkin

/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


pandas.core.frame.DataFrame

In [2]:
user = ps.read_json('/opt/data/initial_load/filtered_user.json')

/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_json`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [3]:
user_df = user

In [8]:
from tiny_functions import * 

In [14]:
user_df['friends_number']

/opt/spark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


0       342
1       418
2       228
3       441
4       267
5       258
6         4
7        38
8       803
9        17
10      379
11       73
12      128
13      110
14      100
15       51
16      331
17      328
18      357
19       32
20      135
21       17
22      608
23      460
24       68
25       74
26      127
27       72
28      589
29      146
30       62
31      969
32      581
33       10
34      318
35       94
36      111
37      104
38       73
39      130
40       88
41      202
42      192
43      673
44      120
45      283
46      188
47       83
48       38
49       10
50      133
51       18
52       45
53       62
54      643
55      378
56      104
57      432
58       15
59       15
60       63
61      210
62       15
63        2
64       51
65      280
66       52
67       13
68       10
69       45
70      410
71       21
72       67
73       41
74       21
75       13
76       66
77      388
78       63
79      923
80      114
81        5
82      144
83  

In [13]:
user_df['friends_number'] = user_df.friends.apply(lambda x: len(x.split(', ')))

/opt/spark/python/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/opt/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
